In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
os.chdir('/home/joe/testbed/DeepSurvivalMachines')

In [4]:
from dsm import datasets
x, t, e = datasets.load_dataset('PBC', sequential = True)

In [5]:
import numpy as np
import seaborn as sns
import pandas as pd

In [6]:
import torch
import tqdm
import torch.nn as nn
import torch.nn.functional as F

# reusing compression code via gumbel

In [8]:
import utils

In [9]:
bs = 128 # batch size

In [10]:
torch.cuda.is_available()

True

In [11]:
(x_train, t_train, e_train,
            x_val, t_val, e_val) = utils.prepocess_training_data_from_lists(x, t, e, vsize=0.2, val_data=None, random_state=563)

In [12]:
print(x_train.shape, t_train.shape, e_train.shape)

torch.Size([250, 16, 25]) torch.Size([250, 16, 1]) torch.Size([250, 16, 1])


In [13]:
scm_model = utils.SurvivalClusteringMachine(x_train.shape[2], hidden=10, 
                k=4, lstm_layers=1, layers=3, dist='Weibull',
               temp=1000., discount=1.0, optimizer='Adam',
               risks=1)

TypeError: object of type 'int' has no len()

In [93]:
xrep = scm_model.represent(x_train)

In [101]:
xrep.contiguous().view(-1, 10).shape

torch.Size([4000, 10])

In [98]:
scm_model.shapeg['1']

Sequential(
  (0): Linear(in_features=20, out_features=4, bias=True)
)

In [102]:
scm_model.shapeg['1'](xrep.contiguous().view(-1, 10))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (4000x10 and 20x4)

In [95]:
#shape, scale, logits = model.forward(x, risk)
shape, scale, logits = scm_model.forward(x_train, '1')

RuntimeError: mat1 and mat2 shapes cannot be multiplied (4000x10 and 20x4)

In [18]:
print(shape.shape, scale.shape, logits.shape)

torch.Size([250, 4]) torch.Size([250, 4]) torch.Size([250, 4])


In [19]:
utils._conditional_weibull_loss(scm_model, x_train, t_train, e_train)

torch.Size([250, 4]) torch.Size([250, 4, 16, 250])


RuntimeError: The size of tensor a (4) must match the size of tensor b (250) at non-singleton dimension 3

In [72]:
xrep = scm_model.represent(x_train)

In [74]:
xrep.shape

torch.Size([250, 16, 10])

In [67]:
t = t_train

lossf = []
losss = []

k_ = shape
b_ = scale
for g in range(scm_model.k):

    k = k_[:, g]
    b = b_[:, g]

    s = - (torch.pow(torch.exp(b)*t, torch.exp(k)))
    f = k + b + ((torch.exp(k)-1)*(b+torch.log(t)))
    f = f + s

    lossf.append(f)
    losss.append(s)

losss = torch.stack(losss, dim=1)
lossf = torch.stack(lossf, dim=1)

lossg = nn.Softmax(dim=1)(logits)

In [57]:
lossg.shape

torch.Size([250, 4])

In [54]:
t_train[0].shape

torch.Size([16, 1])

In [29]:
lossg.shape

torch.Size([250, 4])

In [68]:
lossf.shape

torch.Size([250, 4, 16, 250])

In [59]:
uncens = np.where(e_train.cpu().data.numpy() == int('1'))[0]
cens = np.where(e_train.cpu().data.numpy() != int('1'))[0]

In [2]:
def _conditional_weibull_loss(model, x, t, e, elbo=True, risk='1'):

  alpha = model.discount
  shape, scale, logits = model.forward(x, risk)

  k_ = shape
  b_ = scale

  lossf = []
  losss = []

  for g in range(model.k):

    k = k_[:, g]
    b = b_[:, g]

    s = - (torch.pow(torch.exp(b)*t, torch.exp(k)))
    f = k + b + ((torch.exp(k)-1)*(b+torch.log(t)))
    f = f + s

    lossf.append(f)
    losss.append(s)

  losss = torch.stack(losss, dim=1)
  lossf = torch.stack(lossf, dim=1)

  if elbo:

    lossg = nn.Softmax(dim=1)(logits)
    losss = lossg*losss
    lossf = lossg*lossf
    losss = losss.sum(dim=1)
    lossf = lossf.sum(dim=1)

  else:

    lossg = nn.LogSoftmax(dim=1)(logits)
    losss = lossg + losss
    lossf = lossg + lossf
    losss = torch.logsumexp(losss, dim=1)
    lossf = torch.logsumexp(lossf, dim=1)

  uncens = np.where(e.cpu().data.numpy() == int(risk))[0]
  cens = np.where(e.cpu().data.numpy() != int(risk))[0]
  ll = lossf[uncens].sum() + alpha*losss[cens].sum()

  return -ll/float(len(uncens)+len(cens))

In [33]:
lossg

tensor([[0.2500, 0.2500, 0.2500, 0.2500],
        [0.2500, 0.2500, 0.2500, 0.2500],
        [0.2500, 0.2500, 0.2500, 0.2500],
        [0.2500, 0.2500, 0.2500, 0.2500],
        [0.2500, 0.2500, 0.2500, 0.2500],
        [0.2500, 0.2500, 0.2500, 0.2500],
        [0.2500, 0.2500, 0.2500, 0.2500],
        [0.2500, 0.2500, 0.2500, 0.2500],
        [0.2500, 0.2500, 0.2500, 0.2500],
        [0.2500, 0.2500, 0.2500, 0.2500],
        [0.2500, 0.2500, 0.2500, 0.2500],
        [0.2500, 0.2500, 0.2500, 0.2500],
        [0.2500, 0.2500, 0.2500, 0.2500],
        [0.2500, 0.2500, 0.2500, 0.2500],
        [0.2500, 0.2500, 0.2500, 0.2500],
        [0.2500, 0.2500, 0.2500, 0.2500],
        [0.2500, 0.2500, 0.2500, 0.2500],
        [0.2500, 0.2500, 0.2500, 0.2500],
        [0.2501, 0.2500, 0.2500, 0.2500],
        [0.2500, 0.2500, 0.2500, 0.2500],
        [0.2500, 0.2500, 0.2500, 0.2500],
        [0.2500, 0.2500, 0.2500, 0.2500],
        [0.2500, 0.2500, 0.2500, 0.2500],
        [0.2500, 0.2500, 0.2500, 0

In [3]:
utils.train_scm(scm_model, x_train, t_train, e_train)

NameError: name 'utils' is not defined

In [31]:
scm_model.gate['1'](xrep).shape

torch.Size([250, 4])

In [32]:
scm_model.shapeg['1'](xrep).shape

torch.Size([250, 4])

In [9]:




def add_inputs(input1, input2):
    input_added = input1 + input2
    return input1 + input2 

def divide_inputs(input3, input4):
    input_added = input3 / input4
    return input_added



In [13]:

add_result = add_inputs(4, 6)

divide_result = divide_inputs(14, 7)

print(add_result)
print(divide_result)

10
2.0


In [14]:
add_inputs(1, 2)

3

5


In [28]:
list1 = [4, 6, 1, 7, 0, 15]

In [34]:
list1[2]

1

In [39]:
len(list1)

6

In [ ]:
1 + 2 + 3 + 4 + 5 + 6

In [ ]:
runningtotal = 0

runningtotal = runningtotal + 1
runningtotal = runningtotal + 2
runningtotal = runningtotal + 3
runningtotal = runningtotal + 4
runningtotal = runningtotal + 5

In [ ]:
sumlist = 0

for i in range(6):
    sumlist = sumlist + i

print(var)

In [35]:
list1[0] + list1[1] + list1[2] + list1[3] + list1[4] + list1[5]

33

In [ ]:
runningtotal = 0

runningtotal = runningtotal + list1[0]
runningtotal = runningtotal + list1[1]
runningtotal = runningtotal + list1[2]
runningtotal = runningtotal + list1[3]
runningtotal = runningtotal + list1[4]
runningtotal = runningtotal + list1[5]
